<a href="https://colab.research.google.com/github/hudada369/fy2020-repo-config/blob/master/Pytorch06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**权值初始化：正确的权值初始化可以加速模型的收敛，不恰当的权值初始化导致输出层的输出过大或者过小，最终导致梯度爆炸或者消失，使得模型无法训练**

例如：
1. 适用于饱和激活函数 tanh 等的 Xavier 初始化方法
2. 非饱和激活函数 relu 等的 Kaiming 初始化方法 

1. 权值初始化（梯度消失与爆炸，Xavier 方法与 Kaiming 方法，十种初识化方法）
2. 损失函数（损失函数，代价函数，目标函数这哥仨不是一回事，交叉熵损失，NLL/BCE/BCEWithLogits Loss）

# **权值初始化**网络模型搭建完成之后，对网络中的权重进行合适的初始化
1. 深层神经网络的易发问题就是梯度消失和爆炸，所以针对问题权重初始化，

In [2]:
import torch
import torch.nn as nn
class MLP(nn.Module):
 def __init__(self, neural_num, layers):
  super(MLP, self).__init__()
  self.linears = nn.ModuleList([nn.Linear(neural_num, neural_num, bias=False) 
    for i in range(layers)])
  self.neural_num = neural_num
 # 正向传播
 def forward(self, x):
  for (i, linear) in enumerate(self.linears):
   x = linear(x)

   print("layer:{}, std:{}".format(i, x.std()))
   if torch.isnan(x.std()):
    print('output is nan in {} layers'.format(i))
    break
  
  return x
 # 权值初始化，我们这里使用标准正态
 def initialize(self):
  for m in self.modules():
   if isinstance(m, nn.Linear):
    nn.init.normal_(m.weight.data)      # normal: mean=0, std=1



# 用一下网络
layer_nums = 100
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()

inputs = torch.randn((batch_size, neural_nums))  # normal: mean=0, std=1

output = net(inputs)
print(output)
# 在第30层的时候，网络出现了问题，导致后面输出的值太大了

layer:0, std:15.464187622070312
layer:1, std:248.79202270507812
layer:2, std:3981.748291015625
layer:3, std:62104.64453125
layer:4, std:1001984.1875
layer:5, std:15866013.0
layer:6, std:257165616.0
layer:7, std:4131297024.0
layer:8, std:67165462528.0
layer:9, std:1073208492032.0
layer:10, std:17150222794752.0
layer:11, std:277488139763712.0
layer:12, std:4368071129038848.0
layer:13, std:7.071678255752806e+16
layer:14, std:1.1158861734121308e+18
layer:15, std:1.8216131405388186e+19
layer:16, std:2.8830750029533046e+20
layer:17, std:4.611566110087309e+21
layer:18, std:7.344116739745056e+22
layer:19, std:1.1866482776690813e+24
layer:20, std:1.8891628812221223e+25
layer:21, std:3.0410158581784997e+26
layer:22, std:4.8333825304800914e+27
layer:23, std:7.80522082574331e+28
layer:24, std:1.2387710246706379e+30
layer:25, std:2.0252804219255128e+31
layer:26, std:3.175353145540048e+32
layer:27, std:5.005994929232633e+33
layer:28, std:8.056324618431252e+34
layer:29, std:1.3202718858379784e+36
lay

更改：在进行前向出传播的时候，由于不同层之间方差的影响，使得值方差变大，
1. 目的：让每一层的输入方差都是1，更改的代码，在对每一层权重初始化的时候
nn.init.normal_(m.weight.data, std=np.sqrt(1/self.neural_num))         # 把权重方差改了

2. 「所以我们只要采用恰当的权值初始化方法，就可以实现多层神经网络的输出值的尺度维持在一定范围内, 这样在反向传播的时候，就有利于缓解梯度消失或者爆炸现象的发生」

In [ ]:
def initialize(self):
  for m in self.modules():
    if isinstance(m, nn.Linear):
      nn.init.normal_(m.weight.data, std=np.sqrt(1/self.neural_num))        
 # 把权重方差改了

只在单纯的线性网络中，没有考虑激活函数，如果加上激活函数，例如
tanh，发现值之间的方差越来越小了，如果层数增加，可能引发梯度消失
# **加入激活函数的初始化**
「方差一致性」：保持数据尺度范围维持在恰当范围，通常方差为1
1. 函数nn.init.calculate_gain(nonlinearity, param=None)这个函数的作用是计算激活函数的「方差变化尺度」
  1. 变化尺度：输入数据的方差除以经过激活函数之后的输出数据的方差
  2. nonlinearity 表示激活函数的名称
  3. param 表示激活函数的参数，如 Leaky ReLU 的negative_slop
  4. 「所以Xavier权重初始化，有利于缓解带有sigmoid，tanh的这样的饱和激活函数的神经网络的梯度消失和爆炸现象。」
2. 由于非饱和激活函数relu的出现，Xavier初始化对于relu就不好使了


In [ ]:
def initialize(self):
  for m in self.modules():
    if isinstance(m, nn.Linear):
      # Xavier初始化权重
      tanh_gain = nn.init.calculate_gain('tanh')
      nn.init.xavier_uniform_(m.weight.data, gain=tanh_gain)

In [ ]:
# releu
def initialize(self):
  for m in self.modules():
    if isinstance(m, nn.Linear):
      nn.init.kaiming_normal_(m.weight.data)
      # nn.init.normal_(m.weight.data, std=np.sqrt(2 / self.neural_num))     
# 这两句话其实作用一样，不过自己写还得计算出标准差

十种权重初始化方法
1. 针对饱和激活函数（sigmoid， tanh）：Xavier 均匀分布，Xavier 正态分布
2. 针对非饱和激活函数（relu 及变种）：Kaiming 均匀分布，Kaiming 正态分布
3. 三个常用的分布初始化方法：均匀分布，正态分布，常数分布
4. 三个特殊的矩阵初始化方法：正交矩阵初始化，单位矩阵初始化，稀疏矩阵初始化：

# **损失函数** 衡量模型输出与真实标签的差异
1. 损失函数 计算一个样本的一个差异（loss）
2. 代价函数  计算整个训练集Loss的一个平均值（对于样本集loss的平均值）
3. 目标函数 在代价函数上面加入正则化等等（过拟合和欠拟合之间进行一个权衡）

loss函数也是继承于module类，也有8个参数字典，里面设置一个reduction的参数，函数内部使用的放大也是调用forward方法

交叉熵损失：
nn.CrossEntropyLoss: nn.LogSortmax() 与 nn.NLLLoss() 结合，进行交叉熵计算

1. nn.CrossEntropyLoss(wegiht=None,size_average=None,ignore_index=-100,reduce=None,reduction=mean)
  1. weight：各类别的 loss 设置权值(如果类别不均衡的时候这个参数很有必要了)
  2. ignore_index：忽略某个类别(表示某个类别不去计算 loss)
  3. reduction：计算模式，可为 none/sum/mean，none 表示逐个元素计算，这样有多少个样本就会返回多少个 loss。sum 表示所有元素的 loss 求和，返回标量，mean 所有元素的 loss 求加权平均（加权平均的含义下面会提到），返回标量

2. 先用nn.LogSoftmax()把模型的输出值归一化成了概率分布的形式，然后是单个样本的输出，并且没有求和符号 （交叉熵的损失函数样子是求和符合，现在外面没有有何符号，移动到里面去，-log(exp(x[class] / sigmaexp(x[j])))） x是输出的概况v之，class是类别，在括号内执行了softmax ，把某个神经元的输出归一化概率取值

In [3]:
import torch.nn as nn
# fake data
inputs = torch.tensor([[1, 2], [1, 3], [1, 3]], dtype=torch.float) 
 # 这里就是模型预测的输出， 这里是两个类，可以看到模型输出是数值，我们得softmax一下转成分布
target = torch.tensor([0, 1, 1], dtype=torch.long) 
 # 这里的类型必须是long， 两个类0和1

# 三种模式的损失函数
loss_f_none = nn.CrossEntropyLoss(weight=None, reduction='none')
loss_f_sum = nn.CrossEntropyLoss(weight=None, reduction='sum')
loss_f_mean = nn.CrossEntropyLoss(weight=None, reduction='mean')

# forward
loss_none = loss_f_none(inputs, target)
loss_sum = loss_f_sum(inputs, target)
loss_mean = loss_f_mean(inputs, target)

# view
print("Cross Entropy Loss:\n ", loss_none, loss_sum, loss_mean)

## 结果：
# Cross Entropy Loss:
#   tensor([1.3133, 0.1269, 0.1269]) tensor(1.5671) tensor(0.5224)


Cross Entropy Loss:
  tensor([1.3133, 0.1269, 0.1269]) tensor(1.5671) tensor(0.5224)


当我们对类别加上权重之后，损失就会加倍
「mean 模式下求平均不是除以样本的个数，而是样本所占的权值的总份数」

# **对nn.NLLLoss：实现负对数似然函数里面的负号功能**
1. nn.NLLLoss(weight,size_average=None,ignore_index=-100,reduce=None,reduction=mean)

# **nn.BCELoss** 交叉熵损失函数的特例，二分类交叉熵
1. nn.BCELoss() 参数和上面一致
  1. 不同点： target，这里可以发现和交叉熵那里的标签就不一样了，首先是类型是 float，每个样本属于哪一类的时候要写成独热的那种形式，这是因为看损失函数的计算公式也能看到，每个神经元一一对应的去计算 loss，而不是一个整的神经元向量去计算 loss，看结果也会发现有 8 个 loss，因为每个神经元都一一去计算 loss，根据 inputs，这里是两个神经元的

In [4]:
import torch.nn as nn
# fake data
inputs = torch.tensor([[1, 2], [2, 2], [3, 4],[4,5]], dtype=torch.float) 

target = torch.tensor([[1,0],[1,0],[0,1],[0,1]], dtype=torch.float) 

# sigmoid inpouts要经过sigmoid转换到0-1之间，否则没法比
inputs = torch.sigmoid(inputs)

# 三种模式的损失函数
loss_f_none = nn.BCELoss(weight=None, reduction='none')
loss_f_sum = nn.BCELoss(weight=None, reduction='sum')
loss_f_mean = nn.BCELoss(weight=None, reduction='mean')

# forward
loss_none = loss_f_none(inputs, target)
loss_sum = loss_f_sum(inputs, target)
loss_mean = loss_f_mean(inputs, target)

# view
print("Cross Entropy Loss:\n ", loss_none, loss_sum, loss_mean)


Cross Entropy Loss:
  tensor([[0.3133, 2.1269],
        [0.1269, 2.1269],
        [3.0486, 0.0181],
        [4.0181, 0.0067]]) tensor(11.7856) tensor(1.4732)


3. nn.BCEWithLogitsLoss,结合了 Sigmoid 与二分类交叉熵，注意事项：网络最后不加sigmoid函数
  1. 参数多个一个 pos_wegiht=None,这个是平衡正负样本的权值用的， 对正样本进行一个权值设定。比如我们正样本有 100 个，负样本有 300 个，那么这个数可以设置为 3，在类别不平衡的时候可以用
  2. 相比于二分类的损失函数多了 
  l = y*log(sigmoid(x)) + (1-y)*log(1-sigmoid(x))

3.3 剩余的损失函数
1. nn.L1Loss  计算inputs与target之差的绝对值
2. nn.MSE 回归问题，计算inputs与target之差的平方
3. nn.SmoothL1Loss  平滑的L1Loss
4. nn.PoissonNLLLoss  泊松分布的负对数似然损失函数，分类里面如果发现数据的类别服从泊松分布，可以使用这个损失函数
5. nn.KLDivLoss 功能：计算 KLD， KL 散度，相对熵，注意：需要提前将输入计算 log-probabilities，如通过 nn.logsoftmax()
6. nn.MarginRankingLoss 计算两个向量之间的相似度，用于排序任务。特别说明，该方法计算两组数据之间的差异，也就是每个元素两两之间都会计算差异，返回一个 n*n 的 loss 矩阵。类似于相关性矩阵那种
7. nn.MultiLabelMarginLoss  多标签边界损失函数， 这是一个多标签分类，就是一个样本可能属于多个类，和多分类任务还不一样（多标签问题）
8. nn.SoftMarginLoss  功能：计算二分类的 logistic 损失（二分类问题）
9. nn.MultiLabelSortMarginLoss 功能：SoftMarginLoss 多标签版本 （多标签问题）
10. nn.MultiMarginLoss（hingLoss） 功能：计算多分类的折页损失（多分类问题） 
11. nn.TripletMarginLoss  功能：计算三元组损失，人脸验证中常用
12. nn.HingeEmbeddingLoss 算两个输入的相似性，常用于非线性 embedding 和半监督学习。特别注意，输入的x应为两个输入之差的绝对值， 也就是手动计算两个输入的差值
13. nn.CosineEmbeddingLoss 采用余弦相似度计算两个输入的相似性，常用于半监督学习和 embedding
14. nn.CTCLoss  功能：计算 CTC 损失， 解决时序类数据的分类

1. 「分类问题」
 2. 「二分类单标签问题」：nn.BCELoss, nn.BCEWithLogitsLoss, nn.SoftMarginLoss
 3. 「二分类多标签问题」：nn.MultiLabelSoftMarginLoss
 4. 「多分类单标签问题」: nn.CrossEntropyLoss, nn.NLLLoss, nn.MultiMarginLoss
 5. 「多分类多标签问题」: nn.MultiLabelMarginLoss,
2. 「不常用」：nn.PoissonNLLLoss, nn.KLDivLoss
3. 「回归问题」: nn.L1Loss, nn.MSELoss, nn.SmoothL1Loss
4. 「时序问题」：nn.CTCLoss
5. 「人脸识别问题」：nn.TripletMarginLoss
6. 「半监督Embedding问题(输入之间的相似性)」: nn.MarginRankingLoss, nn.HingeEmbeddingLoss, nn.CosineEmbeddingLoss